# Transformer 학습/추론

In [1]:
# 패키지 import
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim

import torchtext

# 난수 시드 설정
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

## 데이터로더 및 Transformer 모델 준비

In [2]:
from utils.dataloader import get_IMDb_DataLoaders_and_TEXT

train_dl, val_dl, test_dl, TEXT = get_IMDb_DataLoaders_and_TEXT(max_length=256, batch_size=64)

dataloaders_dict = {"train":train_dl, "val":val_dl}

from utils.transformer import TransformerClassification

# 모델 구축
net = TransformerClassification(
    text_embedding_vectors=TEXT.vocab.vectors, d_model=300, max_seq_len=256, output_dim=2)

# 네트워크 초기화
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        # Liner층 초기화
        nn.init.kaiming_normal_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0.0)


# 훈련 모드로 설정
net.train()

# TransformerBlock 모듈을 초기화
net.net3_1.apply(weights_init)
net.net3_2.apply(weights_init)


print('네트워크 설정 완료')


네트워크 설정 완료


In [3]:
criterion = nn.CrossEntropyLoss()

learning_rate = 2e-5
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

## 훈련 및 검증 함수의 구현과 실행

In [4]:
# 모델을 학습시키는 함수를 작성
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPU가 사용 가능한지 확인
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("사용 장치: ", device)
    print('-----start-------')
    # 네트워크를 GPU로
    net.to(device)

    # 네트워크가 어느 정도 고정되면, 고속화시킨다
    torch.backends.cudnn.benchmark = True

    # epoch 루프
    for epoch in range(num_epochs):
        # epoch별 훈련 및 검증 루프
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # 모델을 훈련 모드로
            else:
                net.eval()   # 모델을 검증모드로

            epoch_loss = 0.0  # epoch의 손실합
            epoch_corrects = 0  # epoch의 정답수

            # 데이터 로더에서 미니 배치를 꺼내는 루프
            for batch in (dataloaders_dict[phase]):
                # batch는 Text와 Lable의 사전 오브젝트

                # GPU가 사용 가능하면 GPU로 데이터를 보낸다
                inputs = batch.Text[0].to(device)  # 문장
                labels = batch.Label.to(device)  # 라벨

                # optimizer 초기화
                optimizer.zero_grad()

                # 순전파(forward) 계산
                with torch.set_grad_enabled(phase == 'train'):

                    # mask 작성
                    input_pad = 1  # 단어 ID에 있어서, '<pad>': 1이므로
                    input_mask = (inputs != input_pad)

                    # Transformer에 입력
                    outputs, _, _ = net(inputs, input_mask)
                    loss = criterion(outputs, labels)  # 손실 계산

                    _, preds = torch.max(outputs, 1)  # 라벨을 예측

                    # 훈련시에는 역전파
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # 결과 계산
                    epoch_loss += loss.item() * inputs.size(0)  # loss의 합계를 갱신
                    # 정답수의 합계를 갱신
                    epoch_corrects += torch.sum(preds == labels.data)

            # epoch별 loss와 정답률
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs,
                                                                           phase, epoch_loss, epoch_acc))

    return net


In [5]:
num_epochs = 10
net_trained = train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

사용 장치:  cuda:0
-----start-------
Epoch 1/10 | train |  Loss: 0.5963 Acc: 0.6646
Epoch 1/10 |  val  |  Loss: 0.4431 Acc: 0.8004
Epoch 2/10 | train |  Loss: 0.4439 Acc: 0.7970
Epoch 2/10 |  val  |  Loss: 0.3887 Acc: 0.8304
Epoch 3/10 | train |  Loss: 0.4078 Acc: 0.8156
Epoch 3/10 |  val  |  Loss: 0.3744 Acc: 0.8404
Epoch 4/10 | train |  Loss: 0.3862 Acc: 0.8276
Epoch 4/10 |  val  |  Loss: 0.3597 Acc: 0.8498
Epoch 5/10 | train |  Loss: 0.3710 Acc: 0.8383
Epoch 5/10 |  val  |  Loss: 0.3512 Acc: 0.8550
Epoch 6/10 | train |  Loss: 0.3579 Acc: 0.8425
Epoch 6/10 |  val  |  Loss: 0.3549 Acc: 0.8520
Epoch 7/10 | train |  Loss: 0.3502 Acc: 0.8468
Epoch 7/10 |  val  |  Loss: 0.3434 Acc: 0.8578
Epoch 8/10 | train |  Loss: 0.3438 Acc: 0.8500
Epoch 8/10 |  val  |  Loss: 0.3522 Acc: 0.8530
Epoch 9/10 | train |  Loss: 0.3376 Acc: 0.8518
Epoch 9/10 |  val  |  Loss: 0.3563 Acc: 0.8536
Epoch 10/10 | train |  Loss: 0.3290 Acc: 0.8588
Epoch 10/10 |  val  |  Loss: 0.3686 Acc: 0.8506


In [6]:
# device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net_trained.eval()   # 모델을 검증모드로
net_trained.to(device)

epoch_corrects = 0  # epoch의 정답수

for batch in (test_dl):  # test 데이터의 DataLoader
    # batch는 Text와 Lable의 사전 오브젝트
    
    # GPU가 사용 가능하면 GPU로 데이터를 보낸다
    inputs = batch.Text[0].to(device)  # 문장
    labels = batch.Label.to(device)  # 라벨

    # 순전파(forward) 계산
    with torch.set_grad_enabled(False):

        # mask 작성
        input_pad = 1  # 단어 ID에 있어서, '<pad>': 1이므로
        input_mask = (inputs != input_pad)

        # Transformer에 입력
        outputs, _, _ = net_trained(inputs, input_mask)
        _, preds = torch.max(outputs, 1)  # 라벨을 예측

        # 결과 계산
        # 정답수의 합계를 갱신
        epoch_corrects += torch.sum(preds == labels.data)

# 정답률
epoch_acc = epoch_corrects.double() / len(test_dl.dataset)

print('테스트 데이터 {}개의 정답률: {:.4f}'.format(len(test_dl.dataset),epoch_acc))


테스트 데이터 25000개의 정답률: 0.8385


## Attention 시각화로 판정근거 살피기


In [9]:
# HTML 작성 함수 구현
def highlight(word, attn):
    "Attention 값이 크면 문자 배경을 진한 빨간색으로 하는 html을 출력하는 함수"

    html_color = '#%02X%02X%02X' % (
        255, int(255*(1 - attn)), int(255*(1 - attn)))
    return '<span style="background-color: {}"> {}</span>'.format(html_color, word)


def mk_html(index, batch, preds, normlized_weights_1, normlized_weights_2, TEXT):
    "HTML 데이터를 작성한다"

    # index의 결과를 추출
    sentence = batch.Text[0][index]  # 문장
    label = batch.Label[index]  # 라벨
    pred = preds[index]  # 예측

    # index의 Attention을 추출하고 규격화
    attens1 = normlized_weights_1[index, 0, :]  # 0번째의 <cls>의 Attention
    attens1 /= attens1.max()

    attens2 = normlized_weights_2[index, 0, :]  # 0번째의 <cls>의 Attention
    attens2 /= attens2.max()

    # 라벨 및 예측 결과를 문자로 치환
    if label == 0:
        label_str = "Negative"
    else:
        label_str = "Positive"

    if pred == 0:
        pred_str = "Negative"
    else:
        pred_str = "Positive"

    # 표시용 HTML 작성
    html = '정답 라벨: {}<br>추론 라벨: {}<br><br>'.format(label_str, pred_str)

    # 첫번째 단의 Attention
    html += '[TransformerBlock의 첫번째 단의 Attention을 시각화]<br>'
    for word, attn in zip(sentence, attens1):
        html += highlight(TEXT.vocab.itos[word], attn)
    html += "<br><br>"

    # 두번째 단의 Attention
    html += '[TransformerBlock의 두번째 단의 Attention을 시각화]<br>'
    for word, attn in zip(sentence, attens2):
        html += highlight(TEXT.vocab.itos[word], attn)

    html += "<br><br>"

    return html


In [10]:
from IPython.display import HTML

# Transformer로 처리

# 미니 배치 준비
batch = next(iter(test_dl))

# GPU가 사용 가능하면 GPU로 데이터를 보낸다
inputs = batch.Text[0].to(device)  # 문장
labels = batch.Label.to(device)  # 라벨

# mask 작성
input_pad = 1  # 단어 ID에 있어서, '<pad>': 1이므로
input_mask = (inputs != input_pad)

# Transformer에 입력
outputs, normlized_weights_1, normlized_weights_2 = net_trained(
    inputs, input_mask)
_, preds = torch.max(outputs, 1)  # 라벨을 예측


index = 3  # 출력할 데이터
html_output = mk_html(index, batch, preds, normlized_weights_1,
                      normlized_weights_2, TEXT)  # HTML 작성
HTML(html_output)  # HTML 형식으로 출력


In [11]:
index = 61
html_output = mk_html(index, batch, preds, normlized_weights_1, normlized_weights_2, TEXT)
HTML(html_output)